In [ ]:
# Steps partially taken from https://debuggercafe.com/advanced-facial-keypoint-detection-with-pytorch/

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import torch
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import train_test_split
import timm


%matplotlib inline
%config Completer.use_jedi = False

ROOT = "/home/lenin/code/hat_on_the_head/"
DATA = ROOT + "data/"
KP_DATA = DATA + "kaggle_keypoints/"
RANDOM_SEED = 42

In [ ]:
!ls {KP_DATA}

In [ ]:
def load(path):
    img = cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


def valid_keypoints_plot(image, outputs, orig_keypoints, epoch):
    """
    This function plots the regressed (predicted) keypoints and the actual 
    keypoints after each validation epoch for one image in the batch.
    """
    # detach the image, keypoints, and output tensors from GPU to CPU
    image = image.detach().cpu()
    outputs = outputs.detach().cpu().numpy()
    orig_keypoints = orig_keypoints.detach().cpu().numpy()
    # just get a single datapoint from each batch
    img = image[0]
    output_keypoint = outputs[0]
    orig_keypoint = orig_keypoints[0]
    img = np.array(img, dtype='float32')
    img = np.transpose(img, (1, 2, 0))
    plt.imshow(img)
    
    output_keypoint = output_keypoint.reshape(-1, 2)
    orig_keypoint = orig_keypoint.reshape(-1, 2)
    for p in range(output_keypoint.shape[0]):
        plt.plot(output_keypoint[p, 0], output_keypoint[p, 1], 'r.')
        plt.plot(orig_keypoint[p, 0], orig_keypoint[p, 1], 'b.')
        
    plt.show()
    


def load_item(root, row):
    img = load(root + row.img)
    keypoints = [[float(e) for e in t.split(",")] for t in row.our_kpts.split(";")]
    return img, keypoints

def show(row):
    img, kpts = load_item(KP_DATA, row)
    show_w_kpts(img, kpts)

In [ ]:
df = pd.read_csv(KP_DATA + "training_frames_keypoints.csv")

if True:
    def join_kpts(row):
        kpts = []
        kpts.append(f'{row["78"]},{row["79"]}')
        kpts.append(f'{row["84"]},{row["85"]}')
        return ";".join(kpts)

    df["img"] = df["Unnamed: 0"]
    df["our_kpts"] = df.apply(join_kpts,axis=1)
    df = df.drop(columns=[str(i) for i in range(136)] + ["Unnamed: 0"])

    df.to_csv(DATA + "our_train_kaggle_keypoints.csv", index=False)

print(f"total images {len(df)}")    
df.head()

In [ ]:
def show_w_kpts(img, kpts):
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    keypoints = np.array(kpts)
    for j in range(len(keypoints)):
        plt.plot(keypoints[j, 0], keypoints[j, 1], 'b.')
    plt.show()

tfms = A.Compose([
    A.RandomRotate90(),
    A.LongestMaxSize(448),
    A.ShiftScaleRotate(border_mode=0, value=0),
    A.RandomBrightnessContrast(p=0.2),
], keypoint_params=A.KeypointParams(format='xy'))
tfms_torch = A.Compose([
    A.RandomRotate90(),
    A.LongestMaxSize(448),
    A.ShiftScaleRotate(border_mode=0, value=0),
    A.RandomBrightnessContrast(p=0.2),
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy'))


sample = df.sample(1).iloc[0]
orig_img, orig_kpts = load_item(KP_DATA + "training/", sample)

res = tfms(image=orig_img, keypoints=orig_kpts)
img_tfmd = res["image"]
kpts_tfmd = res["keypoints"]
#show_w_kpts(img_tfmd, kpts_tfmd)

#show_w_kpts(orig_img, orig_kpts)

# Dataset

In [ ]:
class KeypointsDataset(torch.utils.data.Dataset):
    
    def __init__(self, root, df, aug=A.Compose([])):
        self.root = root
        self.df = df
        self.aug = aug

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, i):
        row = self.df.iloc[i]
        orig_img, orig_kpts = load_item(self.root, row)
        res = self.aug(image=orig_img, keypoints=orig_kpts)
        img_tfmd = res["image"]
        kpts_tfmd = res["keypoints"]
        
        return {
            'image': img_tfmd,
            'keypoints': torch.FloatTensor([i for x in kpts_tfmd for i in x])
        }

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
batch_size = 64
num_workers = 0

train_ds = KeypointsDataset(root=KP_DATA + "training/", df=train_df, aug=tfms_torch)
val_ds = KeypointsDataset(root=KP_DATA + "training/", df=val_df, aug=tfms_torch)

train_dl = torch.utils.data.DataLoader(
    dataset=train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
)

val_dl = torch.utils.data.DataLoader(
    dataset=val_ds,
    batch_size=batch_size*2,
    shuffle=False,
    num_workers=num_workers,
)

# Model

In [ ]:
model = timm.create_model('efficientnet_b0', pretrained=True)
model.classifier = torch.nn.Linear(model.classifier.in_features, out_features=4, bias=True)

# Pytorch Lightning

In [ ]:
import pytorch_lightning as pl

In [ ]:
pl.seed_everything(RANDOM_SEED)

In [ ]:
class HatModule(pl.LightningModule):
    def __init__(self, model, optimizer_name, optimizer_hparams):
        super().__init__()
        # Exports the hyperparameters to a YAML file, and create "self.hparams" namespace
        self.save_hyperparameters()
        # Create model
        self.model = model
        # Create loss module
        self.loss_module = torch.nn.SmoothL1Loss()
        # Example input for visualizing the graph in Tensorboard
        # self.example_input_array = torch.zeros((1, 3, 32, 32), dtype=torch.float32)

    def forward(self, imgs):
        # Forward function that is run when visualizing the graph
        return self.model(imgs)

    def configure_optimizers(self):
        # We will support Adam or SGD as optimizers.
        if self.hparams.optimizer_name == "Adam":
            # AdamW is Adam with a correct implementation of weight decay (see here
            # for details: https://arxiv.org/pdf/1711.05101.pdf)
            optimizer = torch.optim.AdamW(self.model.parameters(), **self.hparams.optimizer_hparams)
        elif self.hparams.optimizer_name == "SGD":
            optimizer = torch.optim.SGD(self.model.parameters(), **self.hparams.optimizer_hparams)
        else:
            assert False, f'Unknown optimizer: "{self.hparams.optimizer_name}"'

        # We will reduce the learning rate by 0.1 after 100 and 150 epochs
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        # "batch" is the output of the training data loader.
        imgs, keypoints = batch
        preds = self.model(imgs)
        loss = self.loss_module(preds, keypoints)

        self.log("train_loss", loss)
        return loss  # Return tensor to call ".backward" on

    def validation_step(self, batch, batch_idx):
        imgs, keypoints = batch
        preds = self.model(imgs)
        loss = self.loss_module(preds, keypoints)
        # By default logs it per epoch (weighted average over batches)
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        self.validation_step(batch, batch_idx)

In [ ]:
device = "cuda:0"

trainer = pl.Trainer(
    #default_root_dir=os.path.join(CHECKPOINT_PATH, save_name),  # Where to save models
    # We run on a single GPU (if possible)
    gpus=1 if str(device) == "cuda:0" else 0,
    # How many epochs to train for if no patience is set
    max_epochs=180,
    callbacks=[
        pl.callbacks.ModelCheckpoint(
            save_weights_only=True, mode="min", monitor="val_loss", verbose=True,
        ),  # Save the best checkpoint based on the maximum val_acc recorded. Saves only weights and not optimizer
        pl.callbacks.LearningRateMonitor("epoch"),
    ],  # Log learning rate every epoch
    progress_bar_refresh_rate=1,
) 

module = HatModule(model, 'Adam', {"lr": 0.001})

In [ ]:
trainer.fit(module, train_dataloaders=[train_dl], val_dataloaders=[val_dl])